In [1]:
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score

from scipy.stats import chi2_contingency

from pipelines import preprocessor, preprocessor2
from preprocess import extra_features

In [20]:
train=pd.read_csv('Datasets/Train.csv')
train=extra_features(train)
X=train.drop('cost_category', axis=1)
y= train['cost_category']

<IPython.core.display.Javascript object>

In [3]:
col_drop=[]

for col in X.columns:
    ct=pd.crosstab(y, X[col])
    chi2, p, _, _ =chi2_contingency(ct)
    
    if p >= 0.05 and chi2 <= 3000:
        col_drop.append(col)
        

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
X.drop(columns=col_drop, inplace=True)

In [21]:
X=preprocessor2(X)

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)
len(le.classes_)

6

In [8]:
cat_features=np.where(X.dtype != ('int', 'float'))
cat_features

<IPython.core.display.Javascript object>

(array([0], dtype=int64),)

In [23]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test =train_test_split(X, y, test_size=0.2, random_state=42) 

In [24]:
model=CatBoostClassifier(iterations=10,
                        max_depth=8,
                        learning_rate=0.1,
                        use_best_model=True,
                        eval_metric='Accuracy',
                        )

model.fit(X_train, y_train, eval_set=(X_test,y_test))

0:	learn: 0.5250608	test: 0.5083739	best: 0.5083739 (0)	total: 649ms	remaining: 5.84s
1:	learn: 0.5313429	test: 0.5124257	best: 0.5124257 (1)	total: 1.23s	remaining: 4.94s
2:	learn: 0.5312753	test: 0.5140465	best: 0.5140465 (2)	total: 1.81s	remaining: 4.22s
3:	learn: 0.5463388	test: 0.5267423	best: 0.5267423 (3)	total: 2.39s	remaining: 3.58s
4:	learn: 0.5498514	test: 0.5283630	best: 0.5283630 (4)	total: 2.96s	remaining: 2.96s
5:	learn: 0.5512699	test: 0.5307942	best: 0.5307942 (5)	total: 3.52s	remaining: 2.35s
6:	learn: 0.5541070	test: 0.5321448	best: 0.5321448 (6)	total: 4.09s	remaining: 1.75s
7:	learn: 0.5576196	test: 0.5375473	best: 0.5375473 (7)	total: 4.7s	remaining: 1.17s
8:	learn: 0.5589705	test: 0.5388979	best: 0.5388979 (8)	total: 5.28s	remaining: 586ms
9:	learn: 0.5582275	test: 0.5388979	best: 0.5388979 (8)	total: 5.87s	remaining: 0us

bestTest = 0.538897893
bestIteration = 8

Shrink model to first 9 iterations.


In [25]:
y_pred=model.predict(X_test)

In [26]:
print(classification_report(y_pred, y_test, zero_division=True))

              precision    recall  f1-score   support

           0       0.32      0.49      0.39       502
           1       0.74      0.58      0.65      1219
           2       0.00      1.00      0.00         0
           3       0.00      1.00      0.00         0
           4       0.57      0.52      0.54       543
           5       0.69      0.53      0.60      1438

    accuracy                           0.54      3702
   macro avg       0.39      0.69      0.36      3702
weighted avg       0.64      0.54      0.58      3702

